In [1]:
import os
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-12-02 04:46:51--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  2.04MB/s    in 0.4s    

2022-12-02 04:46:52 (2.04 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"), sep="\t", header=True)# Read in the data from an S3 Bucket
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [5]:
df.count()

3514942

In [6]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = df[["review_id", "customer_id", "product_parent", "review_date"]]
review_id_df.show()

+--------------+-----------+--------------+-----------+
|     review_id|customer_id|product_parent|review_date|
+--------------+-----------+--------------+-----------+
| RAB23OVFNCXZQ|   36075342|     339193102| 2015-08-31|
|R3NORADVJO6IE6|   42462164|     907684644| 2015-08-31|
|R299F4SO98S5OO|   21241933|     752246352| 2015-08-31|
|R2DA9DOT03UW6I|   52570308|     105401756| 2015-08-31|
|R2OGCH681EQHU6|   38200102|     728471129| 2015-08-31|
| R2JMKIC16MHD7|   34866169|     962286893| 2015-08-31|
|R1DB5DA7CWWTI8|     184627|     267002949| 2015-08-31|
|R1N8XWFDK4QACP|    2975964|     570789093| 2015-08-31|
|R19VFLX6MRU0CX|   40946484|     389524802| 2015-08-31|
| RLTD8YDGTP9PT|   12554469|      21425394| 2015-08-31|
| R78B5LCFJ52SR|   35335277|     816815445| 2015-08-31|
|R33SKWL0HEQIQ9|   44957003|     192225755| 2015-08-31|
|R1HIOV0UX7P13D|     444125|     849414056| 2015-08-31|
| R4WS0E0MSP9DH|   35829199|     651316983| 2015-08-31|
| R2RUIDUMDKG7P|   16825098|     694793259| 2015

In [7]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df[["product_id", "product_title"]]
products_df = products_df.drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B002OKKCO8|06-09 KAWASAKI ZX...|
|B00OZXP3BY|Oem Factory Stock...|
|B00HA26HZS|Suzuki Hayabusa G...|
|B00BITHKD2|PT Auto Warehouse...|
|B000688VNE|Hi-Lift Jack Hi-L...|
|B004GCP4GS|OE Replacement Au...|
|B0006HPA02|MSD 32829 8.5mm S...|
|B00EK6NS22|uxcell® Car Bike ...|
|B00WMR1I64|MICTUNING mt00880...|
|B009S31VVW|Dorman 81702 Ford...|
|B00QJ8FA70|Parts Player New ...|
|B00HZOCCBY|MAXFLOORMAT Floor...|
|B00DZPC5NM|Dodge Grand Carav...|
|B001VKT9IW|HushMat 10301 Ult...|
|B009SEJI66|2011-2013 Toyota ...|
|B007QBKODO|ACDelco 18A2729 P...|
|B007LIBJPO|Custom Accessorie...|
|B009YKQR2C|Zone Tech 4pcs 20...|
|B00062YAZ8|Edelbrock 4201 El...|
|B008IAHBWY|Jeep Wrangler Tj ...|
+----------+--------------------+
only showing top 20 rows



In [8]:
customers_df = df.groupBy("customer_id").count()
customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    2583392|    4|
|     129005|    1|
|    3962209|    2|
|   16092109|    1|
|   46841294|    1|
|   42126527|    4|
|   47138302|    1|
|   10484128|    2|
|    5427648|    1|
|     580770|    1|
|   14286306|    6|
|   13880860|    1|
|   24095000|    2|
|   50189080|    1|
|   50583551|    4|
|   15562874|    1|
|   51068983|    3|
|   10245571|    6|
|   28932014|    8|
|   42891605|    4|
+-----------+-----+
only showing top 20 rows



In [9]:
vine_df = df[["review_id", "star_rating", "helpful_votes", "total_votes", "vine"]]
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RAB23OVFNCXZQ|          1|            0|          0|   N|
|R3NORADVJO6IE6|          5|            0|          0|   N|
|R299F4SO98S5OO|          5|            1|          1|   N|
|R2DA9DOT03UW6I|          5|            2|          3|   N|
|R2OGCH681EQHU6|          5|            0|          0|   N|
| R2JMKIC16MHD7|          5|            2|          2|   N|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|
|R1N8XWFDK4QACP|          5|            0|          0|   N|
|R19VFLX6MRU0CX|          5|            0|          0|   N|
| RLTD8YDGTP9PT|          1|            0|          0|   N|
| R78B5LCFJ52SR|          4|            0|          0|   N|
|R33SKWL0HEQIQ9|          5|            0|          0|   N|
|R1HIOV0UX7P13D|          5|            0|          0|   N|
| R4WS0E0MSP9DH|          5|            

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS

In [ ]:
# Write products_df to table in RDS

In [ ]:
# Write customers_df to table in RDS


In [ ]:
# Write vine_df to table in RDS